# Agents
- What is it?
    - LangChain Agents are like smart assistants for language models. They figure out what you want, use the right tools to get the information or complete a task, and then tell you the results.
    - LangChain agents are like intelligent decision-makers that leverage language models (LLMs) to understand your requests and determine the best course of action. Unlike LLMs alone, which primarily generate text, agents can interact with external resources like APIs, databases, or even the command line through the use of specialized tools.
- Why is it important?
    - They're important because language models can't do everything on their own. Agents help them access information, perform actions, and solve complex problems they couldn't handle alone.
## Types of Agents
- Zero-shot-React-Description (Used in the Example):
    - How it Works: The agent receives a description of the available tools and the user's request. It then decides which tool to use and how to format the input for the tool in a single step (zero-shot).
    - When to Use: Great for simple tasks where the agent can make decisions quickly without needing to interact with the tools multiple times.
- ReAct:
    - How it Works: The agent iteratively interacts with the tools, thinking about the user's request and the results of each tool call to decide its next action.
    - When to Use: Suitable for more complex tasks that require multiple steps or where the optimal tool might not be immediately clear.
- Conversational ReAct:
    - How it Works: Similar to ReAct, but specifically designed for conversational settings. It can handle multi-turn conversations and maintain context.
    - When to Use: Ideal for building chatbots or virtual assistants where maintaining the conversation's flow is crucial.


- Additional information about ReAct:
    - ReAct stands for Reasoning and Acting. It is a prompting technique designed to enhance the reasoning capabilities of large language models (LLMs). The ReAct approach involves breaking down a complex task into smaller steps, where the LLM reasons about the current state, selects an action to take, observes the outcome of that action, and then repeats the process until the task is completed.



- Rule of thumb to figure out which agent to use:
    - If the task is simple and can be done in one step, use Zero-shot-React-Description.
    - If the task is more complex and requires multiple steps or decisions, use ReAct.
    - If the task involves a conversation or multiple interactions with the user, use Conversational ReAct.


In [ ]:
# Basic Example
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

# Define your OpenAI API key
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'

# Create a very simple tool
def get_current_time():
    """Returns the current time in H:MM AM/PM format."""
    import datetime
    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

tools = [
    Tool(
        name = "Time",
        func=get_current_time,
        description="useful for when you need to know the current time"
    ),
]
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)

# Initialize an agent
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Test the agent
response = agent.run("What time is it?")
print(response) 


In [ ]:
# Comparison Example   
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import ConversationalChatAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper

# 1. Define Tools
# Define your OpenAI API key
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

def get_weather(location: str) -> str:
    """Gets the current weather in a given location."""
    from langchain.utilities import OpenWeatherMapAPIWrapper

    weather_api = OpenWeatherMapAPIWrapper()
    weather = weather_api.run(location)
    return weather

def get_fun_fact(subject: str) -> str:
    """Gets a fun fact about a given subject using SerpAPI."""
    search = SerpAPIWrapper()
    params = {
        "engine": "google",
        "q": f"fun fact about {subject}",
    }
    return search.run(params)

tools = [
    Tool(
        name="GetWeather",
        func=get_weather,
        description="useful for when you need to know the weather in a given location"
    ),
    Tool(
        name="GetFunFact",
        func=get_fun_fact,
        description="useful for when you want to find a fun fact about a given subject"
    )
]

# 2. Initialize Zero-Shot and ReAct Agents
query = "What's the weather in London, and tell me a fun fact about the city?"
print("Zero-Shot Agent:")
agent_zero = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
agent_zero.run(query)

print("\nReAct Agent:")
agent_react = initialize_agent(
    tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True
)
agent_react.run(query)

# 3. Initialize Conversational Agent
memory = ConversationBufferMemory(memory_key="chat_history")
agent_conversational = ConversationalChatAgent.from_llm_and_tools(llm, tools, memory=memory)

# 4. Run the Conversational Agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent_conversational, tools=tools, verbose=True)
print("\nConversational ReAct Agent:")
agent_executor.run("What's the weather like in London today?")  
agent_executor.run("That's interesting. Now, can you tell me a fun fact about London?") 


Zero-Shot Agent's Thoughts:

Prompt: "What's the weather in London, and tell me a fun fact about the city?"

Reasoning:

Analyze the query: The query has two parts: one about weather and one about a fun fact.
Tool Selection: I can use both the GetWeather tool for the weather information and the GetFunFact tool for the fun fact.
Execute Tools: I'll run both tools simultaneously, one to fetch the weather in London and the other to search for a fun fact about London.
Combine Results: Once I have the results from both tools, I'll combine them into a single response.
ReAct Agent's Thoughts:

Prompt: "What's the weather in London, and tell me a fun fact about the city?"

Reasoning:

Analyze the query: The query has two parts, but I'll tackle them one by one.
First Task: I need to find the weather in London, so I'll use the GetWeather tool.
Execute GetWeather: [Calls the tool and receives the weather information]
Second Task: Now I need to find a fun fact about London. I'll use the GetFunFact tool.
Execute GetFunFact: [Calls the tool and receives a fun fact]
Combine Results: I'll combine the weather information and the fun fact into a single response.
Conversational ReAct Agent's Thoughts:

Prompt 1: "What's the weather like in London today?"

Reasoning:

Analyze the query: The user wants to know about the weather in London.
Tool Selection: I'll use the GetWeather tool.
Execute GetWeather: [Calls the tool and receives the weather information]
Formulate Response: I'll tell the user the current weather in London.
Store Information: I'll remember that we were talking about London for the next turn.
Prompt 2: "That's interesting. Now, can you tell me a fun fact about London?"

Reasoning:

Analyze the query: The user wants to know a fun fact about London.
Recall Context: I remember from the previous turn that we were talking about London.
Tool Selection: I'll use the GetFunFact tool, since I already know the location.
Execute GetFunFact: [Calls the tool and receives a fun fact about London]
Formulate Response: I'll tell the user the fun fact I found.

# Tools
- What is it?
    - Think of tools as specialized actions your language model can take to interact with the world beyond just text.  They are like different skills or abilities you can give your LLM to make it more useful and versatile.
- Examples of Tools:
    - Search Tool: Looks up information on the internet (like Google Search).
    - Calculator: Performs mathematical calculations.
    - Database Lookup: Queries a database to retrieve specific information.
    - Code Execution: Runs code snippets and returns the results.
    - Image Generation: Creates images based on text descriptions.
- Why use tools:
    - Access to Real-World Information: LLMs are trained on static data, but tools can get them up-to-date info from the web or other sources.
    - Perform Actions: Tools let LLMs do things besides just generate text, making them more interactive and helpful.
    - Solve Complex Problems: Many problems require multiple steps, like looking up info, doing calculations, and then summarizing the results. Tools help break these down.


In [ ]:
from langchain.tools import BaseTool

class MyCalculatorTool(BaseTool):
    name = "Calculator"
    description = "Useful for when you need to perform mathematical calculations."

    def _run(self, query: str) -> str:
        """Use the calculator."""
        try:
            result = eval(query)  # Be careful with `eval` in production!
            return str(result)
        except Exception as e:
            return f"Error: {e}"


What's happening under the hood with this example when working with agents:

This is where agents come in. The agent acts as the "brain" that decides which tool to use based on your input. Here's how they work together:

You: "What's 25 times 12?"
Agent: (Thinks) "This sounds like a math problem. I'll use the Calculator tool."
Agent (to Calculator): "25 * 12"
Calculator: "300"
Agent (to You): "The answer is 300."

- LangChain Tools: https://python.langchain.com/v0.1/docs/integrations/tools/

    - Search Engines:
        - SerpAPIWrapper (requires API key)
        - TavilySearchResults (requires API key)
        - DuckDuckGoSearchRun
        - FireCrawl
        - TODO: Show output for each tool in the list for same website.
    - Wikipedia:

        - WikipediaQueryRun
    - Code Execution:

        - PythonREPLTool
    - Other:

        - WolframAlphaAPIWrapper (requires API key)
        - RequestsGetTool (for making HTTP GET requests)
        - Many more in langchain_community.tools

In [ ]:
# Simple tool constructor

from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper
import random

def recommend_movie(genre: str) -> str:
    """Recommends a movie based on genre using SerpAPI."""
    search = SerpAPIWrapper()
    params = {
        "engine": "google",
        "q": f"best {genre} movies"
    }
    results = search.run(params)  
    # Extract movie titles from the search results (this is simplified for the demo)
    movies = [title for title in results.split("\n") if title]  
    return random.choice(movies) if movies else "No movies found for that genre."

movie_recommender_tool = Tool(
    name="MovieRecommender",
    func=recommend_movie,
    description="Useful for when you want to get a movie recommendation for a specific genre."
)

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool
from langchain.llms import OpenAI

# Define your OpenAI API key
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

@tool("Current Time", "Get the current time in your location.")
def get_current_time() -> str:
    """
    This tool returns the current time in your location.
    """
    from datetime import datetime

    now = datetime.now()
    return f"Current time is {now.strftime('%Y-%m-%d %H:%M:%S')}"

tools = [get_current_time]  # The decorator automatically creates the Tool object
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What time is it?")
print(response)

- Tool Constructor Approach:

    - When you need fine-grained control over input validation.
    - When you want to pass multiple arguments of different types to the tool.
    - When your tool's logic is more complex and might benefit from a separate function definition.

- @tool Decorator Approach:

    - When you want to quickly create a simple tool with a single string input and string/dictionary output.
    - When you plan to share your tool on LangChainHub.
    - When you prefer concise and self-documenting code.

In [ ]:
from langchain.tools import BaseTool
import random

class MovieRecommenderTool(BaseTool):
    name = "MovieRecommender"
    description = "Useful for recommending movies based on a genre. Input should be the genre you are interested in."

    def _run(self, genre: str) -> str:
        """Run the movie recommender."""
        movies = {
            "Action": ["Die Hard", "Mad Max: Fury Road"],
            "Comedy": ["21 Jump Street", "The Hangover"],
            "Sci-Fi": ["Blade Runner", "Interstellar"],
            # ... (Add more genres and movies here)
        }
        if genre in movies:
            return random.choice(movies[genre])
        else:
            return "I'm not familiar with that genre yet."


In [ ]:
from langchain.tools import BaseTool
from pydantic import Field, BaseModel
import random

# Define the Pydantic model for input validation
# Helps agent to determine parameters, types, and any validation rules
class MovieRecommenderInput(BaseModel):
    genre: str = Field(description="The genre of movie you're interested in.")

# Define the MovieRecommender tool
class MovieRecommenderTool(BaseTool):
    name = "MovieRecommender"
    description = "Useful for recommending movies based on a genre."
    args_schema: MovieRecommenderInput  # Use the Pydantic model for input validation

    def _run(self, genre: str) -> str:
        """Run the movie recommender."""
        # A larger movie dataset for demonstration purposes (you could replace with API calls)
        movies = {
            "Action": ["Die Hard", "Mad Max: Fury Road", "John Wick", "The Raid: Redemption"],
            "Comedy": ["21 Jump Street", "The Hangover", "Booksmart", "Bridesmaids"],
            "Sci-Fi": ["Blade Runner", "Interstellar", "Arrival", "Dune"],
            "Drama": ["The Shawshank Redemption", "The Godfather", "Parasite", "12 Angry Men"],
            "Horror": ["The Shining", "Get Out", "Hereditary", "A Quiet Place"],
        }

        if genre.lower() in movies:  # Case-insensitive genre matching
            return random.choice(movies[genre.lower()])
        else:
            return "I'm not familiar with that genre yet. Try a different one!"

# Example usage with an agent (not necessary for the tool itself)
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI

# Define your OpenAI API key
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

tools = [MovieRecommenderTool()]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("Recommend a good sci-fi movie.")


In [ ]:
from langchain.tools import BaseTool
from pydantic import Field, BaseModel
import random

# Define the Pydantic model for input validation
class MovieRecommenderInput(BaseModel):
    genre: str = Field(description="The genre of movie you're interested in.")

# Define the Pydantic model for output (two movie recommendations)
class MovieRecommenderOutput(BaseModel):
    primary_recommendation: str = Field(description="The primary movie recommendation.")
    backup_recommendation: str = Field(description="An alternative movie recommendation.")

# Define the MovieRecommender tool
class MovieRecommenderTool(BaseTool):
    name = "MovieRecommender"
    description = """
    Useful for recommending movies based on a genre. 
    Input should be a single word representing the genre (e.g., "action", "comedy", "sci-fi").
    """
    args_schema: MovieRecommenderInput    # Input validation
    return_direct: bool = True            # Return as dictionary 

    def _run(self, genre: str) -> MovieRecommenderOutput:
        """Run the movie recommender."""
        movies = {
            "Action": ["Die Hard", "Mad Max: Fury Road", "John Wick", "The Raid: Redemption"],
            "Comedy": ["21 Jump Street", "The Hangover", "Booksmart", "Bridesmaids"],
            "Sci-Fi": ["Blade Runner", "Interstellar", "Arrival", "Dune"],
            "Drama": ["The Shawshank Redemption", "The Godfather", "Parasite", "12 Angry Men"],
            "Horror": ["The Shining", "Get Out", "Hereditary", "A Quiet Place"],
        }

        genre = genre.lower()  # Case-insensitive genre matching
        if genre in movies and len(movies[genre]) >= 2:
            # Sample two movies randomly, ensuring they are different
            primary, backup = random.sample(movies[genre], 2)
        elif genre in movies:
            primary = random.choice(movies[genre])
            backup = "I couldn't find another movie in that genre, sorry!"
        else:
            primary = backup = "I'm not familiar with that genre yet. Try a different one!"

        # Return the recommendations as a MovieRecommenderOutput object (which will be converted to a dict)
        return MovieRecommenderOutput(primary_recommendation=primary, backup_recommendation=backup)

# Example usage
tool = MovieRecommenderTool()
result = tool.run("sci-fi")
print(result) 


LangChain Tools Cheat Sheet
What are Tools?
Think of tools as superpowers for your language models (LLMs). Tools let your LLM interact with the world beyond just text. They can search the web, perform calculations, access databases, run code, and much more.

Why Use Tools?
Give your LLM access to up-to-date information (it only knows what it was trained on).
Enable your LLM to take actions (not just generate text).
Solve complex tasks that require multiple steps.

3 ways to create tools:
- Tool Constructor Approach: For fine-grained control and complex logic.
- @tool Decorator Approach: For quick and simple tools.
- Pydantic Validation: For input validation and type checking.

Using tools with agents:

`tools = [my_tool, another_tool, ...]`


Initializing agents:
```python
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
```

Run it:
`agent.run("Your query here")`
